# Fine-tuning a BERT model for text extraction with the SQuAD dataset

We have fine-tuned [BERT implemented by HuggingFace](https://huggingface.co/bert-base-uncased) for the text-extraction task with a dataset of questions and answers with the [SQuAD (The Stanford Question Answering Dataset)](https://rajpurkar.github.io/SQuAD-explorer/) dataset. Let evaluate the model.

This notebook is based on [BERT (from HuggingFace Transformers) for Text Extraction](https://keras.io/examples/nlp/text_extraction_with_bert/).

More info:
- [Glossary - HuggingFace docs](https://huggingface.co/transformers/glossary.html#model-inputs)
- [BERT NLP — How To Build a Question Answering Bot](https://towardsdatascience.com/bert-nlp-how-to-build-a-question-answering-bot-98b1d1594d7b)

In [1]:
import os
import utility.data_processing as dpp
import utility.testing as testing
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertForQuestionAnswering
from tokenizers import BertWordPieceTokenizer

/apps/daint/UES/6.0.UP04/sandboxes/sarafael/hpcpython2022/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets.utils import disable_progress_bar
from datasets import disable_caching


disable_progress_bar()
disable_caching()

In [3]:
hf_model = 'bert-base-uncased'
bert_cache = os.path.join(os.getcwd(), 'cache')
save_path = os.path.join(bert_cache, f'{hf_model}-tokenizer')

In [4]:
# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer(os.path.join(save_path, 'vocab.txt'),
                                   lowercase=True)

In [5]:
model = BertForQuestionAnswering.from_pretrained(
    hf_model,
    cache_dir=os.path.join(bert_cache, f'{hf_model}_qa')
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [6]:
model_path_name = f'/scratch/snx3000/class401/bert_trained_deepspeed_example'

# load the model on cpu
model.load_state_dict(
    torch.load(model_path_name,
               map_location=torch.device('cpu'))
)

# load the model on gpu
# model.load_state_dict(torch.load(model_path_name))

<All keys matched successfully>

In [7]:
hf_dataset = load_dataset('squad')

Reusing dataset squad (/users/class424/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [8]:
val_ds = hf_dataset['validation'].flatten()

In [9]:
max_len = 384

In [10]:
processed_val_ds = val_ds.map(
    lambda example: dpp.process_squad_item_batched(example, max_len, tokenizer),
    remove_columns=val_ds.column_names,
    batched=True,
    num_proc=12
)

In [11]:
processed_val_ds.set_format(type='torch')

In [12]:
batch_size = 1

eval_dataloader = DataLoader(
    processed_val_ds,
    shuffle=False,
    batch_size=batch_size
)

In [13]:
squad_example_objects = []
for item in val_ds:
    squad_examples = dpp.squad_examples_from_dataset(item, max_len, tokenizer)
    try:
        squad_example_objects.extend(squad_examples)
    except TypeError:
        squad_example_objects.append(squad_examples)
        
assert len(processed_val_ds) == len(squad_example_objects)

In [14]:
start_sample = 24100
num_test_samples = 10
for i, eval_batch in enumerate(eval_dataloader):
    if i > start_sample:
        testing.EvalUtility(eval_batch, [squad_example_objects[i]], model).results()

    if i > start_sample + num_test_samples:
        break

─────────────────────── {'exact_match': 0.0, 'f1': 44.44444444444445} ───────────────────────

❓ What popular phrase is associated with the Doctor Who series?

🤖 The phrase "Hiding behind (or 'watching from behind') the sofa" entered British pop 
culture, signifying in humour the stereotypical early-series behaviour of children who wanted
to avoid seeing frightening parts of a television programme while remaining in the room to 
watch the remainder of it. The phrase retains this association with Doctor Who, to the point 
that in 1991 the Museum of the Moving Image in London named their exhibition celebrating the 
programme "Behind the Sofa". The electronic theme music too was perceived as eerie, novel, 
and frightening, at the time. A 2012 article placed this childhood juxtaposition of fear and 
thrill "at the center of many people's relationship with the show", and a 2011 online vote at
Digital Spy deemed the series the "scariest TV show of all time".

✅ Hiding behind (or 'watching from behind') the sofa

─────────────────────── {'exact_match': 0.0, 'f1': 44.44444444444445} ───────────────────────

...

✅ scariest TV show of all time

──────────────────────────── {'exact_match': 100.0, 'f1': 100.0} ────────────────────────────

❓ What did people vote the Doctor Who series as in a 2011 online vote?

🤖 The phrase "Hiding behind (or 'watching from behind') the sofa" entered British pop 
culture, signifying in humour the stereotypical early-series behaviour of children who wanted
to avoid seeing frightening parts of a television programme while remaining in the room to 
watch the remainder of it. The phrase retains this association with Doctor Who, to the point 
that in 1991 the Museum of the Moving Image in London named their exhibition celebrating the 
programme "Behind the Sofa". The electronic theme music too was perceived as eerie, novel, 
and frightening, at the time. A 2012 article placed this childhood juxtaposition of fear and 
thrill "at the center of many people's relationship with the show", and a 2011 online vote at
Digital Spy deemed the series the "scariest TV show of all time".

✅ scariest TV show of all time